# Example – Closed Volume

## Introduction

This example aims to introduce more comprehensive ways to define fuel, mixture of fuels and air, as well as finite rate reactions, using a simple geometry. The example is divided in 3 seperate Tasks.


## Task 1  
### Non-predefined fuel

The combustion reaction in FDS is defined by the `REAC` namelist. So far we used fuels, which were pre-defined in FDS. The list of these fuels can be found in Table 14.1. of the user guide {cite}`FDS-UG-6.7.5`. The physical properties of these species are embeded in FDS and do not need to be explicitly specified.

Complete the pre-built FDS input file for this task {download}`pentane_fuel.fds` .
* Use pentane as the fuel. For this purpose, specify the fuel name and chemical formula.
* Specify the values for `SOOT_YIELD`, `CO_YIELD` and `HEAT_OF_COMBUSTION` respectively, 0.01, 0.025 and $\mf 48~kJ/kg$.
* Add slices for pentane and oxygen mass fractions at $\mf y=0.0$ .
* Illustrate the slice data at about $\mf t=3~s$ either with Python or take a snapshot in Smokeview.

### Definitions
**Mass/Volume fraction** of a substance within a mixture is the ratio of its mass/volume to the total mass/volume of the mixture. 

* Mass fraction: $\mf Y_i = m_i/m_T$
* Volume fraction: $\mf X_i = v_i/v_T$


### Solution

In [ ]:
import fdsreader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
Simulation = fdsreader.Simulation('./non-predefined_fuel/')

time = Simulation.slices[0].get_nearest_timestep(3)

plt.imshow(Simulation.slices[0][0].data[time].T, 
               origin = 'lower', 
               extent = Simulation.slices[0].extent.as_list())
        
plt.title('Pentane')
plt.xlabel("x / m")
plt.ylabel("z / m")
plt.colorbar(label = "Mass Fraction")
        
plt.savefig("figs/mass_frac_pentane.svg")
plt.close()

In [ ]:
plt.imshow(Simulation.slices[1][0].data[time].T, 
               origin = 'lower', 
               extent = Simulation.slices[0].extent.as_list())

plt.title('Oxygen')
plt.xlabel("x / m")
plt.ylabel("z / m")
plt.colorbar(label = "Mass Fraction")


plt.savefig("figs/mass_frac_oxygen.svg")
plt.close()

:::{figure-md}

<img src="figs/mass_frac_pentane.svg" width="60%">

Mass fraction of propane at $\mf t=3~s$.

:::

:::{figure-md}

<img src="figs/mass_frac_oxygen.svg" width="60%">

Mass fraction of oxygen at $\mf t=3~s$.

:::

## Task 2
### Mixture Fractions

In this task, we define the reactants and products of a chemical combustion with its stochiometric coefficients in a FDS input file, instead of letting FDS to calculate the coefficients itself.

Given is the propane combustion reaction.

$$
\mf C_3H_8 + 5~[O_2 + (0.79/0.21)N_2] \rightarrow 3 CO_2 + 4 H_2O + 5(0.79/0.21)N_2
$$


* According to the balanced reaction, complete the `VOLUME_FRACTION` of products in the FDS input file {download}`mixture_fraction.fds`. 
* Complete two parameters `SPEC_ID_NU` and `NU` in the `REAC` namelist. Use the user guide for the definitions.
* Define two devices for pentane and oxygen mass fractions. Instead of XB a point probe, use `DB = 'WHOLE DOMAIN'` to measure the parameter in the whole computational domain.

* Execute the simulation.

* Simulate the same scenario with an initial mass fraction of 0.03 and 0.12 for propane. You can use the `INIT` namelist to specify the initial conditions.
* Compare the mass fraction of Propane and Oxygen as well as the time of combustion for all three simulations in Smokeview. 

:::{figure-md}

<img src="figs/initial_mixture.png" width="60%">

Note that the initial condition is not a homogeneous mixture! This slice shows the initial mass fraction of propane.

:::

### Solution

In [ ]:
Sim3 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.03/')
Sim6 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.06/')
Sim12 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.12/')

devc3 = Sim3.devices
devc6 = Sim6.devices
devc12 = Sim12.devices

plt.plot(devc3["Time"].data, devc3['C3H8'].data, label='C3H8 MASS FRACTION, 0.03')
plt.plot(devc6["Time"].data, devc6['C3H8'].data, label='C3H8 MASS FRACTION, 0.06')    
plt.plot(devc12["Time"].data, devc12['C3H8'].data, label='C3H8 MASS FRACTION, 0.12')
    
plt.legend(loc='upper right')
plt.title('C3H8')
plt.xlabel('time / s')
plt.ylabel('Mass Fraction')
plt.grid()

plt.savefig('figs/C3H8_MassFractions.svg')
plt.close()

In [ ]:
Sim3 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.03/')
Sim6 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.06/')
Sim12 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.12/')

devc3 = Sim3.devices
devc6 = Sim6.devices
devc12 = Sim12.devices

plt.plot(devc3["Time"].data, devc3['O2'].data, label='O2 MASS FRACTION, 0.03')
plt.plot(devc6["Time"].data, devc6['O2'].data, label='O2 MASS FRACTION, 0.06')    
plt.plot(devc12["Time"].data, devc12['O2'].data, label='O2 MASS FRACTION, 0.12')
    
plt.legend(loc='upper right')
plt.title('O2')
plt.xlabel('time / s')
plt.ylabel('Mass Fraction')
plt.grid()

plt.savefig('figs/O2_MassFractions.svg')
plt.close()

:::{figure-md}

<img src="figs/C3H8_MassFractions.svg" width="50%">

Mass fraction of propane during the combustion, with three different scenarios for initial mass of Propane.

:::

:::{figure-md}

<img src="figs/O2_MassFractions.svg" width="50%">

Mass fraction of oxygen during the combustion, with three different scenarios for initial mass of Propane.

:::

## Task 3
### Finite rate reaction

By default, FDS uses the infinite reaction rate assumption. However, if needed for a more complex chemical analysis, FDS is capable to compute finite-rate reactions using an Arrhenius model. In this task a hot particle ignites the homogeneous mixture of propane and air. At the end of the combustion, all of the reactants will be consumed.

The Arrhenius equation describes the specific reaction rate as: 

$$
\mf k = A \cdot exp\left(-\frac{E_a}{k_B T}\right) \quad .
$$

The balanced chemical equation for the combustion of propane is:

$$
\mf C_3H_8 + 5 O_2 \rightarrow 3 CO_2 + 4 H_2O \quad .
$$

Using the same geometry as the previous tasks, complete the FDS input file {download}`arrhenius_test.fds` .

* What do `A`, `E` and `N_S` stands for in FDS file `REAC` namelist? For this, check out FDS user guide section 15.3 .
* For each species in the reaction, define a device, to measure the mass fraction during the reaction time in the whole room. 
* Plot the heat release rate.
* Plot all the mass fractions together to see the trend of reactants and products.
* Do reactions in FDS follow mass conservation? Plot the total mass of species during the reaction.

### Solution

In [ ]:
Simulation = fdsreader.Simulation('./reactionrate_arrhenius/')

plt.plot(Simulation.hrr['Time'].data, Simulation.hrr['HRR'].data)
plt.title('HRR')
plt.xlabel('Time / s')
plt.ylabel('Heat Release Rate / kW')
plt.grid()

plt.savefig('figs/arrheniusHRR.svg', bbox_inches='tight')
plt.close()

:::{figure-md}

<img src="figs/arrheniusHRR.svg" width="50%">

Heat release rate.

:::

In [ ]:
devc = Simulation.devices

plt.plot(devc["Time"].data, devc['O2'].data,   label='O2')
plt.plot(devc["Time"].data, devc['C3H8'].data, label='C3H8')
plt.plot(devc["Time"].data, devc['CO2'].data,  label='CO2')
plt.plot(devc["Time"].data, devc['H2O'].data,  label='H2O')

plt.plot(devc["Time"].data, np.around(devc['O2'].data + 
                                      devc['C3H8'].data + 
                                      devc['CO2'].data + 
                                      devc['H2O'].data,3), label='Total mass')
    
plt.legend(loc='upper right')
plt.xlabel('time / s')
plt.ylabel('Mass Fraction')
plt.grid()

plt.savefig("figs/arrhenius_MassFractions.svg")
plt.close()

:::{figure-md}

<img src="figs/arrhenius_MassFractions.svg" width="70%">

Mass fractions of all species during the reaction.

:::